In [24]:
# Practical crypto trading system using CoinGecko API
# Last modified: 2024-06-21  # Updated today

"""
Real-world crypto trading toolkit with CoinGecko API (more for research)
- Can handle API limitations pragmatically
- Includes market-specific adaptations
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os
import requests
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import warnings

# Suppress minor warnings to reduce noise
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
plt.style.use('ggplot')  # Practical choice over fancy styles

# Updated coin mapping with verified IDs
COIN_IDS = {
    'BTC-USD': 'bitcoin',
    'ETH-USD': 'ethereum',
    'LTC-USD': 'litecoin',
    'XRP-USD': 'ripple',
    'ADA-USD': 'cardano',
    'DOGE-USD': 'dogecoin',
    'SOL-USD': 'solana',
    'BNB-USD': 'binancecoin',
    'DOT-USD': 'polkadot',
    'AVAX-USD': 'avalanche-2',
    'MATIC-USD': 'matic-network',
    'SHIB-USD': 'shiba-inu',
    'TRX-USD': 'tron',
    'LINK-USD': 'chainlink',
    'ATOM-USD': 'cosmos'
}

def fetch_crypto_data(assets, start_date, end_date):
    # Fetch historical data with robust rate handling
    print(f"\nFetching data for {len(assets)} coins...")
    datasets = {}
    failed_symbols = []
    api_calls = 0
    last_call_time = 0

    for i, symbol in enumerate(assets):
        coin_id = COIN_IDS.get(symbol)
        if not coin_id:
            print(f"No mapping for {symbol}, skipping")
            failed_symbols.append(symbol)
            continue

        print(f"  {i+1}/{len(assets)}: {symbol}...", end='', flush=True)

        # Smart rate limiting
        current_time = time.time()
        if api_calls > 0 and current_time - last_call_time < 15:
            sleep_time = 20 - (current_time - last_call_time)
            time.sleep(sleep_time)

        start_ts = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
        end_ts = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())
        url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
        params = {'vs_currency': 'usd', 'from': start_ts, 'to': end_ts}

        try:
            response = requests.get(url, params=params, timeout=25)
            api_calls += 1
            last_call_time = time.time()

            if response.status_code == 429:
                print(" (rate limited)", end='')
                # Check if we have rate limit info
                if 'x-ratelimit-remaining' in response.headers:
                    remaining = int(response.headers['x-ratelimit-remaining'])
                    if remaining <= 1:
                        print("Cooling down for 60s...", end='')
                        time.sleep(60)
                        response = requests.get(url, params=params, timeout=25)
                        api_calls += 1
                        last_call_time = time.time()

            if response.status_code != 200:
                print(f"Status {response.status_code}")
                failed_symbols.append(symbol)
                continue

            data = response.json()
            prices = data.get('prices', [])

            if not prices:
                print("Empty price data")
                failed_symbols.append(symbol)
                continue

            # Create dataframe with proper datetime handling
            df = pd.DataFrame(prices, columns=['timestamp', 'price'])
            df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('date', inplace=True)

            # Resample to daily and handle gaps
            daily_df = df.resample('D').agg({
                'price': 'ohlc',
                'timestamp': 'first'
            })
            daily_df.columns = ['Open', 'High', 'Low', 'Close', 'timestamp']
            ohlc = daily_df[['Open', 'High', 'Low', 'Close']]

            # Add volume if available
            if 'total_volumes' in data:
                volumes = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'Volume'])
                volumes['date'] = pd.to_datetime(volumes['timestamp'], unit='ms')
                volumes.set_index('date', inplace=True)
                daily_volumes = volumes['Volume'].resample('D').sum()
                ohlc['Volume'] = daily_volumes

            # Filter to exact date range
            ohlc = ohlc.loc[start_date:end_date]

            if ohlc.empty:
                print("No data after filtering")
                failed_symbols.append(symbol)
                continue

            # Final cleanup
            ohlc = ohlc.ffill().bfill().dropna()

            datasets[symbol] = ohlc
            print(f"Checkmark! {len(ohlc)} days")

        except Exception as e:
            print(f"Failed ({str(e)})")
            failed_symbols.append(symbol)

    if failed_symbols:
        print(f"\n Failed to fetch: {', '.join(failed_symbols)}")
        print("Possible solutions:")
        print("- Try again during off-peak hours (UTC 00:00-06:00)")
        print("- Reduce number of assets or increase sleep time")
        print("- Use a paid API key for higher rate limits")

    return datasets

def generate_signals(data, short=12, long=26):
    # Generate trading signals with position sizing
    signals = pd.DataFrame(index=data.index)

    # EMA calculations
    data['Fast_MA'] = data['Close'].ewm(span=short, adjust=False).mean()
    data['Slow_MA'] = data['Close'].ewm(span=long, adjust=False).mean()

    # Volatility-based position sizing
    daily_vol = data['Close'].pct_change().rolling(14).std().fillna(0)
    position_size = 1 - np.minimum(daily_vol * 10, 0.8)  # Reduce size within high volatility

    signals['Position'] = np.where(data['Fast_MA'] > data['Slow_MA'], position_size, 0)
    signals['Entry'] = signals['Position'].diff().fillna(0)

    return data, signals

def backtest_strategy(data, signals, capital=10000):
    """Run backtest with realistic assumptions"""
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['Asset_Returns'] = data['Close'].pct_change().fillna(0)

    # Strategy returns with proper shift and NaN handling
    portfolio['Strategy_Returns'] = signals['Position'].shift(1).fillna(0) * portfolio['Asset_Returns']

    # Account for trading fees (0.1% per trade)
    trade_count = np.abs(signals['Entry']).sum()
    if trade_count > 0:
        portfolio['Strategy_Returns'] -= trade_count * 0.001 / len(portfolio)

    # Portfolio value calculation
    portfolio['Strategy_Value'] = capital * (1 + portfolio['Strategy_Returns']).cumprod()
    portfolio['Buy_Hold_Value'] = capital * (1 + portfolio['Asset_Returns']).cumprod()

    return portfolio

def plot_results(data, signals, portfolio, symbol, output_dir):
    """Create and save practical visualizations"""
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")

    # Price and signals plot
    plt.figure(figsize=(12, 6))
    plt.plot(data['Close'], label='Price', alpha=0.8, linewidth=1.5)
    plt.plot(data['Fast_MA'], label=f'EMA({12})', linestyle='--', alpha=0.7)
    plt.plot(data['Slow_MA'], label=f'EMA({26})', linestyle='--', alpha=0.7)

    # Plot entry/exit markers
    buys = signals[signals['Entry'] > 0]
    sells = signals[signals['Entry'] < 0]
    plt.scatter(buys.index, data.loc[buys.index, 'Close'],
                marker='^', color='g', s=80, label='Buy', alpha=0.9)
    plt.scatter(sells.index, data.loc[sells.index, 'Close'],
                marker='v', color='r', s=80, label='Sell', alpha=0.9)

    plt.title(f"{symbol} Trading Signals", fontsize=14)
    plt.legend()
    plt.savefig(f"{output_dir}/{symbol}_signals_{timestamp}.png", bbox_inches='tight', dpi=120)
    plt.close()

    # Performance comparison
    plt.figure(figsize=(12, 6))
    plt.plot(portfolio['Strategy_Value'], label='Strategy', linewidth=2)
    plt.plot(portfolio['Buy_Hold_Value'], label='Buy & Hold', linewidth=2)

    # Annotate final values
    final_strat = portfolio['Strategy_Value'].iloc[-1]
    final_bh = portfolio['Buy_Hold_Value'].iloc[-1]
    plt.annotate(f'Strategy: ${final_strat:,.0f}',
                 (portfolio.index[-1], final_strat),
                 xytext=(10, 10), textcoords='offset points',
                 fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="w"))
    plt.annotate(f'B&H: ${final_bh:,.0f}',
                 (portfolio.index[-1], final_bh),
                 xytext=(10, -30), textcoords='offset points',
                 fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="w"))

    plt.title(f"{symbol} Performance Comparison", fontsize=14)
    plt.legend()
    plt.savefig(f"{output_dir}/{symbol}_performance_{timestamp}.png", bbox_inches='tight', dpi=120)
    plt.close()

def calculate_metrics(portfolio):
    # Calculate practical performance metrics
    returns = portfolio['Strategy_Returns']
    bh_returns = portfolio['Asset_Returns']

    # Basic metrics
    if len(portfolio) > 1 and portfolio['Strategy_Value'].iloc[0] > 0:
        total_return = (portfolio['Strategy_Value'].iloc[-1] / portfolio['Strategy_Value'].iloc[0]) - 1
    else:
        total_return = 0

    if len(portfolio) > 1 and portfolio['Buy_Hold_Value'].iloc[0] > 0:
        bh_total_return = (portfolio['Buy_Hold_Value'].iloc[-1] / portfolio['Buy_Hold_Value'].iloc[0]) - 1
    else:
        bh_total_return = 0

    # Risk metrics
    volatility = returns.std() * np.sqrt(252) if len(returns) > 1 else 0
    bh_volatility = bh_returns.std() * np.sqrt(252) if len(bh_returns) > 1 else 0

    # Drawdown calculation
    if len(portfolio) > 1 and not portfolio['Strategy_Value'].empty:
        peak = portfolio['Strategy_Value'].cummax()
        drawdown = (peak - portfolio['Strategy_Value']) / peak
        max_dd = drawdown.max()
    else:
        max_dd = 0

    return {
        'Total Return': f"{total_return:.1%}",
        'Buy&Hold Return': f"{bh_total_return:.1%}",
        'Volatility': f"{volatility:.1%}",
        'B&H Volatility': f"{bh_volatility:.1%}",
        'Max Drawdown': f"{max_dd:.1%}"
    }

if __name__ == "__main__":
    # Configuration
    SYMBOLS = [
        'BTC-USD',
        'ETH-USD',
        'LTC-USD',
        'XRP-USD',
        'ADA-USD',
        'DOGE-USD',
        'SOL-USD',
        'BNB-USD',
        'DOT-USD',
        'AVAX-USD',
        'MATIC-USD',
        'SHIB-USD',
        'TRX-USD',
        'LINK-USD',
        'ATOM-USD'
    ]

    END_DATE = datetime.now().strftime('%Y-%m-%d')
    # 6 months of time
    START_DATE = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')
    # Should be a reasonable test amount
    CAPITAL = 5000
    OUTPUT_DIR = "crypto_trading_results"

    print("\n" + "="*50)
    print("PRACTICAL CRYPTO TRADING ANALYZER")
    print(f"Date Range: {START_DATE} to {END_DATE}")
    print("="*50)

    # Track total time
    start_time = time.time()

    # Fetch data
    data = fetch_crypto_data(SYMBOLS, START_DATE, END_DATE)

    # Process each asset
    for symbol, prices in data.items():
        # Require min data
        if prices.empty or len(prices) < 10:
            print(f"\nSkipping {symbol} - insufficient data")
            continue

        print(f"\n===== Analyzing {symbol} =====")
        print(f"Records: {len(prices)}")
        print(f"First: ${prices['Close'].iloc[0]:,.2f} | " f"Last: ${prices['Close'].iloc[-1]:,.2f} | " f"Change: {(prices['Close'].iloc[-1]/prices['Close'].iloc[0]-1):.1%}")

        # Generate signals
        prices, signals = generate_signals(prices)

        # Backtest
        portfolio = backtest_strategy(prices, signals, CAPITAL)

        # Calculate results
        final_value = portfolio['Strategy_Value'].iloc[-1]
        final_bh = portfolio['Buy_Hold_Value'].iloc[-1]
        print(f"Strategy Final: ${final_value:,.2f}")
        print(f"Buy & Hold Final: ${final_bh:,.2f}")

        # Metrics
        metrics = calculate_metrics(portfolio)
        for k, v in metrics.items():
            print(f"{k}: {v}")

        # Visualization
        plot_results(prices, signals, portfolio, symbol, OUTPUT_DIR)
        print(f"Saved charts to {OUTPUT_DIR}/")

    # Final report
    elapsed = time.time() - start_time
    print("\n" + "="*50)
    print(f"ANALYSIS COMPLETE IN {elapsed:.1f} SECONDS")
    print("="*50)


PRACTICAL CRYPTO TRADING ANALYZER
Date Range: 2024-12-24 to 2025-06-22

Fetching data for 15 coins...
  1/15: BTC-USD...Checkmark! 181 days
  2/15: ETH-USD...Checkmark! 181 days
  3/15: LTC-USD...Checkmark! 181 days
  4/15: XRP-USD...Checkmark! 181 days
  5/15: ADA-USD...Checkmark! 181 days
  6/15: DOGE-USD...Checkmark! 181 days
  7/15: SOL-USD...Checkmark! 181 days
  8/15: BNB-USD...Checkmark! 181 days
  9/15: DOT-USD...Checkmark! 181 days
  10/15: AVAX-USD...Checkmark! 181 days
  11/15: MATIC-USD...Checkmark! 181 days
  12/15: SHIB-USD...Checkmark! 181 days
  13/15: TRX-USD...Checkmark! 181 days
  14/15: LINK-USD...Checkmark! 181 days
  15/15: ATOM-USD...Checkmark! 181 days

===== Analyzing BTC-USD =====
Records: 181
First: $94,644.91 | Last: $101,532.57 | Change: 7.3%
Strategy Final: $5,131.76
Buy & Hold Final: $5,363.87
Total Return: 2.6%
Buy&Hold Return: 7.3%
Volatility: 19.5%
B&H Volatility: 39.4%
Max Drawdown: 10.6%
Saved charts to crypto_trading_results/

===== Analyzing ETH-U